In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [12]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [13]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [14]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [17]:
index_to_char = dict(zip(char_to_index.values(), char_to_index.keys()))

In [18]:
index_to_char

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}

In [19]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [20]:
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [21]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [25]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [26]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [28]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [29]:
net = Net(input_size, hidden_size, output_size)

In [30]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([1, 5, 5])


In [31]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


In [32]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [33]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [34]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)


0 loss:  1.6518566608428955 prediction:  [[2 2 2 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  eeeee
1 loss:  1.4163000583648682 prediction:  [[2 2 2 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  eeee!
2 loss:  1.2031692266464233 prediction:  [[4 4 4 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppp!
3 loss:  0.9822881817817688 prediction:  [[4 4 3 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppll!
4 loss:  0.7469989657402039 prediction:  [[4 4 3 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppll!
5 loss:  0.5436116456985474 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.3894684314727783 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.27038535475730896 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.18306270241737366 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.13040685653686523 prediction:  [[4 4 3 2 0

In [35]:
outputs

tensor([[[-4.8631, -2.0879, -2.1492, -6.2306,  6.2315],
         [-4.3257, -5.0797, -3.7697, -0.5420,  7.1884],
         [-2.6366, -5.2342, -0.5589,  8.3621,  1.9372],
         [-2.6545, -2.4765,  7.3901, -0.9561, -1.2741],
         [ 8.9969, -0.9407, -1.9173,  0.5518, -3.9218]]],
       grad_fn=<AddBackward0>)